In [2]:
import pandas as pd
import numpy as np
import obonet, networkx as nx
from sklearn.preprocessing import MultiLabelBinarizer
import joblib, os

# Ścieżki
PATH_TERMS = "../data/bronze/Train/train_terms.tsv"
PATH_OBO = "../data/bronze/Train/go-basic.obo"

# 1. Propagacja etykiet
go_graph = obonet.read_obo(PATH_OBO)
df_terms = pd.read_csv(PATH_TERMS, sep="\t", names=['Protein_ID', 'term', 'ontology'])

def get_ancestors(go_id, graph):
    return nx.descendants(graph, go_id) if go_id in graph else set()

protein_to_terms = df_terms.groupby('Protein_ID')['term'].apply(list).to_dict()
expanded_data = []

print("🔄 Propagacja etykiet...")
for prot_id, terms in protein_to_terms.items():
    extended_set = set(terms)
    for t in terms:
        extended_set.update(get_ancestors(t, go_graph))
    for final_term in extended_set:
        expanded_data.append({'Protein_ID': prot_id, 'term': final_term})

df_silver = pd.DataFrame(expanded_data)

# 2. Wybór TOP 1500 i Binaryzacja
top_terms = df_silver['term'].value_counts().nlargest(1500).index.tolist()
df_filtered = df_silver[df_silver['term'].isin(top_terms)]
protein_labels = df_filtered.groupby('Protein_ID')['term'].apply(list)

mlb = MultiLabelBinarizer(classes=top_terms)
y = mlb.fit_transform(protein_labels)

# 3. Zapis
os.makedirs("../data/gold", exist_ok=True)
os.makedirs("../models", exist_ok=True)
np.save("../data/gold/y_train_labels.npy", y)
np.save("../data/gold/train_protein_ids.npy", protein_labels.index.values)
joblib.dump(top_terms, "../models/top_terms_1500.pkl")
print(f"✅ Zapisano y: {y.shape} i listę terminów.")

🔄 Propagacja etykiet...
✅ Zapisano y: (82404, 1500) i listę terminów.
